# SO3 to R1 Attractor, SO3 Damping

In [ ]:
using PBDS, StaticArrays, LinearAlgebra, BenchmarkTools, Rotations, Random

## Setup

### Point Distance Attractor

In [ ]:
function PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::DistanceFromPoint{SO3,R1}) where m
    R = reshape(xme, Size(3,3))
    R_center = reshape(task_map.position_center)
    R_diff = R*R_center'
    θ = acos((tr(R_diff)-1)/2)
    d = θ != 0 ? abs(θ) : 0.
end
PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::DistanceFromPoint{SO3,R1}) =
    SA[norm(xme - task_map.position_center)]
PBDS.domain_coord_rep(::DistanceFromPoint{SO3,R1}) = EmbRep()

PBDS.metric_chart(xn, task::Attractor{<:DistanceFromPoint{SO3,R1}}, CN::Chart{1,R1}) = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:DistanceFromPoint{SO3,R1}}, CN::Chart{1,R1}) = xn[1]^2
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:DistanceFromPoint{SO3,R1}}, CN::Chart{1,R1}) =  0*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:DistanceFromPoint{SO3,R1}}, CN::Chart{1,R1}) = 
    default_weight_metric(xn, vn, task, CN)

### Rotation Damping

In [ ]:
PBDS.default_coord_rep(::Damping{<:Identity{SO3,SO3}}) = ChartRep()
PBDS.metric_chart(xn, task::Damping{<:Identity{SO3,SO3}}, CN::Chart{ExpMap0,SO3}) =
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Damping{<:Identity{SO3,SO3}}, CN::Chart{<:ExpMap,SO3}) = 0.

function PBDS.dissipative_forces_chart(xn, vn, task::Damping{<:Identity{SO3,SO3}}, CN::Chart{<:ExpMap,SO3})
    xne, vne = chart_to_emb_differential(xn, vn, CN)
    Fne = -3*vne
    ∂xne_∂xn = chart_to_emb_jacobian(xn, CN)
    Fn = ∂xne_∂xn'*Fne
end
PBDS.weight_metric_chart(xn, vn, task::Damping{<:Identity{SO3,SO3}}, CN::Chart{ExpMap0,SO3}) =
    default_weight_metric(xn, vn, task, CN)
PBDS.home_task_chart(task::Damping{<:Identity{SO3,SO3}}, CN::Chart{<:ExpMap,SO3}) = Chart{ExpMap0,SO3}()

## Point Acceleration

In [ ]:
# Initial state
skew(w) = SA[0.   -w[3]  w[2]
             w[3]  0.   -w[1]
            -w[2]  w[1]  0.]

Random.seed!(8)
R = rand(RotMatrix{3}).mat
ω = R*SA[0.,0.,20.]
xme = reshape(R, Size(9))
vme = reshape(skew(ω)*R, Size(9))

C0 = Chart{PBDS.ExpMap0,SO3}()

M = SO3
tasks, CNs = TaskList(), ChartList()

N = R1
CN = Chart{1,N}()
R_goal = RotX(0.)
xme_goal = reshape(R_goal, Size(9))
push!(tasks, Attractor(DistanceFromPoint{M,N}(xme_goal)))
push!(CNs, CN)

N = SO3
CN = choose_chart_emb(xme, C0)
push!(tasks, Damping(Identity{M,N,Float64}()))
push!(CNs, CN)

CM = CN
robot_coord_rep = EmbRep()
σxddot, = multiple_task_acceleration(xme, vme, tasks, CM, CNs, robot_coord_rep)

## Single Trajectory

In [ ]:
using Plots, Makie, Observables, ProgressMeter

In [ ]:
Time = 10
dt = 0.05

traj = propagate_tasks(xme, vme, tasks, CM, CNs, Time, dt, robot_coord_rep, log_tasks=true)
traj.vm[end]

In [ ]:
function frot(i)
    q = UnitQuaternion(RotMatrix(traj.xm[i]))
    qrot = Quaternion(q.x, q.y, q.z, q.w)
end

iobs = Observable(1)
ax_size, plot_size = 2, 800
limits = FRect3D((-ax_size, -ax_size, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
widths = SA[1.,2.,3.]
rect = Rect(Vec(-widths./2), Vec(widths))
rect_mesh = mesh!(scene, rect, color = :orange; limits)
Makie.rotate!(rect_mesh[end], frot(1))

axis = scene[Axis]
axis.showaxis = false
display(scene)

In [ ]:
function record_scene(scene, filename, iobs, N, framerate=60)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        Makie.rotate!(rect_mesh[end], frot(i))
        next!(p)
    end
    display("text/html", html_video(filename))
end

filename = "SO3ToSO3_Damping.mp4"
record_scene(scene, filename, iobs, length(traj.xm))